In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
!pip install git+https://github.com/indra-ipd/scipy.git

  Cloning https://github.com/indra-ipd/scipy.git to /tmp/pip-req-build-vqp77x2r
  Running command git clone -q https://github.com/indra-ipd/scipy.git /tmp/pip-req-build-vqp77x2r
  Running command git submodule update --init --recursive -q
  From https://github.com/scipy/boost-headers-only
   * branch            2110ce20e59917b85b12059b3ddd8b133549f662 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scipy: filename=scipy-1.7.0.dev0+1032.253cdda-cp37-cp37m-linux_x86_64.whl size=56685333 sha256=49da1b308ace7e7e9803a43d49ec0b642048bf12097418be52b4542f1f09b7be
  Stored in directory: /tmp/pip-ephem-wheel-cache-hpoegiih/wheels/3d/e4/26/f8c192c4555c2f4643b23b020176fb3638f7fd22c1069269d6
Successfully built scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency res

In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import logging, sys
logging.disable(sys.maxsize)

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import collections
import time
from sklearn import datasets

In [4]:
# Import MNIST data (http://yann.lecun.com/exdb/mnist/)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
Xtrain, Ytrain, Xtest, Ytest = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

seed = np.random.randint(100)
tf.set_random_seed(seed)
image_size = 28
chan_num = 1
labels_size = 10
learning_rate = 0.001
batch_size = 128
batches = int(len(Ytrain) / batch_size)
iterations = batches

epoch = 10

algo = ['oLMoQ','oLNAQ','oLBFGS','Adam']
col = {'oNAQ': 'b', 'oMoQ': 'm', 'oBFGS': 'g', 'oLNAQ': 'b', 'oLMoQ': 'm', 'oLBFGS': 'g','Adam': 'r'}
#col = {'oNAQ': 'b', 'oMoQ': 'm', 'oBFGS': 'g', 'oLNAQ': '#00CED1', 'oLMoQ': '#DAA520', 'oLBFGS': '#90EE90','Adam': 'r'}


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
training_data = tf.placeholder(tf.float32, [None, image_size*image_size*chan_num])
labels = tf.placeholder(tf.float32, [None, labels_size])

L1 = tf.contrib.layers.fully_connected(inputs=training_data, num_outputs=100, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm)
L2 = tf.contrib.layers.fully_connected(inputs=L1, num_outputs=50, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm)
#L3 = tf.contrib.layers.fully_connected(inputs=L2, num_outputs=16, activation_fn=tf.nn.sigmoid)
output = tf.contrib.layers.fully_connected(inputs=L2, num_outputs=10, activation_fn=tf.nn.softmax)

#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output))
loss = tf.reduce_mean(-tf.reduce_sum(labels*tf.log(output), reduction_indices=1))

correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [6]:
perm_idx = []
for num in range(500):
    perm_idx.append(np.random.permutation(len(Ytrain)))



def get_batches(x_tr,y_tr, size,ep_num):

    #shuffle data
    #idx = np.random.permutation(len(y_tr))
    idx = perm_idx[ep_num-1]
    x_tr, y_tr = x_tr[idx], y_tr[idx]

    num_batch = int(len(y_tr)/size)
    data = []
    lab = []
    for i in range(num_batch):
        data.append(x_tr[i*size:i*size+size])
        lab.append(y_tr[i * size:i * size + size])
    return data,lab

def update(l, a):
    global train_loss, train_acc
    train_loss = l
    train_acc = a

count = 0

In [ ]:
for meth in algo:
    color = col[meth]
    if meth == 'Adam':
        train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
        epoch = 50
        timePlt = collections.deque(maxlen=iterations * epoch)
        errPlt = collections.deque(maxlen=iterations * epoch)

    else:
        m = 4
        # mu = 0.8
        sk_vec = collections.deque(maxlen=m)
        yk_vec = collections.deque(maxlen=m)
        Hk_mat = collections.deque(maxlen=1)
        alpha_k = collections.deque(maxlen=1)
        mu_val = collections.deque(maxlen=1)

        timePlt = collections.deque(maxlen=iterations * epoch)
        errPlt = collections.deque(maxlen=iterations * epoch)

        alpha_k.append(1)
        vk_vec = collections.deque(maxlen=1)
        vk_vec.append(0)
        dirNorm = True

        if meth == 'oNAQ' or meth == 'oBFGS' or meth == 'oLNAQ' or meth == 'oLBFGS':  # vk_vec=None, sk_vec=None, yk_vec=None, m=8, alpha_k=1.0, mu=None, dirNorm=True,

            train_step = tf.contrib.opt.ScipyOptimizerInterface(
                loss, method=meth.lower(),
                options={'maxiter': iterations, 'disp': False, 'vk_vec': vk_vec, 'sk_vec': sk_vec, 'yk_vec': yk_vec,
                         'timeplot': timePlt, 'err': errPlt, 'Hk_mat':Hk_mat,
                         'm': m, 'alpha_k': alpha_k, 'muk': mu_val, 'dirNorm': dirNorm})

        elif meth == 'oMoQ' or meth == 'oLMoQ':  # vk_vec=None, sk_vec=None, yk_vec=None, m=8, alpha_k=1.0, mu=None, dirNorm=True,
            # grad_curr = collections.deque(maxlen=1)
            # grad_pre = collections.deque(maxlen=1)
            gfk_vec = collections.deque(maxlen=2)

            train_step = tf.contrib.opt.ScipyOptimizerInterface(
                loss, method=meth.lower(),
                options={'maxiter': iterations, 'disp': False, 'vk_vec': vk_vec, 'sk_vec': sk_vec, 'yk_vec': yk_vec,
                         # 'grad_pre': grad_pre, 'grad_curr': grad_curr,
                         'timeplot': timePlt, 'err': errPlt, 'gfk_vec': gfk_vec,'Hk_mat':Hk_mat,
                         'm': m, 'alpha_k': alpha_k, 'muk': mu_val, 'dirNorm': dirNorm})

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    print("Initial Error of ", meth, " : ", sess.run(loss, feed_dict={training_data: Xtrain, labels: Ytrain}))

    step = 0
    alpha_k.append(1)
    eta0 = batch_size / (batch_size + 2)

    test_acc_plot = []
    test_loss_plot = []
    train_acc_plot = []
    train_loss_plot = []

    for ep in range(1, epoch + 1):
        print("EPOCH ", ep, " : ##########")
        theta_k = 1

        alpha_k.append(alpha_k[-1] * 0.9)
        alpha_k.append(np.maximum(alpha_k[-1],0.001))
        # alpha_k.append(0.5)
        Xtr, Ytr = get_batches(Xtrain, Ytrain, batch_size, ep)
        for i in range(iterations):
            step += 1

            data, lab = Xtr[i], Ytr[i]
            feed_dict = {training_data: data, labels: lab}

            if meth == 'Adam':
                start = time.time()
                _, train_loss, train_acc = sess.run([train_step, loss, accuracy], feed_dict=feed_dict)
                end = time.time()
                timePlt.append(end - start)



            else:

                if meth != 'oMoQ' or meth != 'oLMoQ':
                    if step > 1:
                        alpha_k.append(1 / np.sqrt(step))

                    # if meth == 'oMoQ':
                    #  if step > 1:
                    #    alpha_k.append(10 / (10+step))

                if meth == 'oBFGS' or meth == 'oLBFGS':
                    alpha_k.append(eta0 * (10 / (10 + step)))
                    mu_val.append(0)
                    mu = mu_val[-1]

                if meth == 'oNAQ' or meth == 'oLNAQ':

                    mu_val.append(0.8)
                    mu = mu_val[-1]

                if meth == 'oMoQ' or meth == 'oLMoQ':
                    # theta_kp1 = ((1e-5 - (theta_k * theta_k)) + np.sqrt(((1e-5 - (theta_k * theta_k)) * (1e-5 - (theta_k * theta_k))) + 4 * theta_k * theta_k)) / 2
                    # mu = np.minimum((theta_k * (1 - theta_k)) / (theta_k * theta_k + theta_kp1), 0.95)
                    # theta_k = theta_kp1

                    mu_val.append(0.8)
                    mu = mu_val[-1]

                res = train_step.minimize(sess, fetches=[loss, accuracy],
                                          loss_callback=update,
                                          feed_dict=feed_dict)

            test_loss, test_acc = sess.run([loss, accuracy], feed_dict={training_data: Xtest, labels: Ytest})
            test_acc_plot.append(test_acc * 100)
            test_loss_plot.append(test_loss)
            train_acc_plot.append(train_acc * 100)
            train_loss_plot.append(train_loss)

            if i % 50 == 0:
                print(
                    'Step {}; train loss {}; train accuracy {}; test loss {}; test accuracy {}; alpha {}; mu {}'.format(
                        i, train_loss, train_acc * 100, test_loss, test_acc * 100, alpha_k[0], mu))

    leg = algo

    plt.figure(1)
    plt.semilogy(np.cumsum(timePlt), train_loss_plot, color)
    plt.xlabel('Time')
    plt.ylabel('Training Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(2)
    plt.semilogy(np.cumsum(timePlt), test_loss_plot, color)
    plt.xlabel('Time')
    plt.ylabel('Test Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(3)
    plt.plot(np.cumsum(timePlt), train_acc_plot, color)
    plt.ylim((70, 100))
    plt.xlabel('Time')
    plt.ylabel('Train Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(4)
    plt.plot(np.cumsum(timePlt), test_acc_plot, color)
    plt.ylim((70, 100))
    plt.xlabel('Time')
    plt.ylabel('Test Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(5)
    plt.semilogy(train_loss_plot, color)
    plt.xlabel('Iteration')
    plt.ylabel('Training Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(6)
    plt.semilogy(test_loss_plot, color)
    plt.xlabel('Iteration')
    plt.ylabel('Test Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(7)
    plt.plot(train_acc_plot, color)
    # plt.ylim((70,100))
    plt.xlabel('Iteration')
    plt.ylabel('Train Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(8)
    plt.plot(test_acc_plot, color)
    plt.ylim((60,100))
    plt.xlabel('Iteration')
    plt.ylabel('Test Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(9)
    plt.semilogy(np.cumsum(timePlt), np.minimum.accumulate(train_loss_plot), color)
    plt.xlabel('Time')
    plt.ylabel('Training Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(10)
    plt.semilogy(np.cumsum(timePlt), np.minimum.accumulate(test_loss_plot), color)
    plt.xlabel('Time')
    plt.ylabel('Test Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(11)
    plt.plot(np.cumsum(timePlt), np.maximum.accumulate(train_acc_plot), color)
    plt.ylim((40, 100))
    plt.xlabel('Time')
    plt.ylabel('Train Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(12)
    plt.plot(np.cumsum(timePlt), np.maximum.accumulate(test_acc_plot), color)
    plt.ylim((40, 100))
    plt.xlabel('Time')
    plt.ylabel('Test Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(13)
    plt.semilogy(np.minimum.accumulate(train_loss_plot), color)
    plt.xlabel('Iteration')
    plt.ylabel('Training Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(14)
    plt.semilogy(np.minimum.accumulate(test_loss_plot), color)
    plt.xlabel('Iteration')
    plt.ylabel('Test Loss')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(15)
    plt.plot(np.maximum.accumulate(train_acc_plot), color)
    plt.ylim((40, 105))
    plt.xlabel('Iteration')
    plt.ylabel('Train Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    plt.figure(16)
    plt.plot(np.maximum.accumulate(test_acc_plot), color)
    plt.ylim((40, 100))
    plt.xlabel('Iteration')
    plt.ylabel('Test Accuracy')
    plt.legend(leg)
    plt.tight_layout()

    timePlt.clear()
    sk_vec.clear()
    yk_vec.clear()
    vk_vec.clear()

    sess.close()

print('seed: ', seed)
plt.show()

Initial Error of  oLMoQ  :  2.6321366
EPOCH  1  : ##########
Parameters:  84060
Step 0; train loss 1.2892625331878662; train accuracy 68.75; test loss 1.664481282234192; test accuracy 44.94999945163727; alpha 0.9; mu 0.8


/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/opt/python/training/external_optimizer.py:402: OptimizeWarning: Unknown solver options: Hk_mat
  result = scipy.optimize.minimize(*minimize_args, **minimize_kwargs)


Step 50; train loss 0.24076008796691895; train accuracy 90.625; test loss 0.23984628915786743; test accuracy 93.11000108718872; alpha 0.14002800840280097; mu 0.8
Step 100; train loss 0.28250157833099365; train accuracy 91.40625; test loss 0.19142597913742065; test accuracy 94.38999891281128; alpha 0.09950371902099892; mu 0.8
Step 150; train loss 0.21201840043067932; train accuracy 93.75; test loss 0.16929002106189728; test accuracy 95.02999782562256; alpha 0.08137884587711594; mu 0.8
Step 200; train loss 0.30750513076782227; train accuracy 89.0625; test loss 0.15589341521263123; test accuracy 95.35999894142151; alpha 0.07053456158585983; mu 0.8
Step 250; train loss 0.19027021527290344; train accuracy 94.53125; test loss 0.1459905505180359; test accuracy 95.7099974155426; alpha 0.06311944030978031; mu 0.8
Step 300; train loss 0.1181061789393425; train accuracy 98.4375; test loss 0.13894134759902954; test accuracy 95.81999778747559; alpha 0.0576390417704235; mu 0.8
Step 350; train loss 0

/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/opt/python/training/external_optimizer.py:402: OptimizeWarning: Unknown solver options: Hk_mat
  result = scipy.optimize.minimize(*minimize_args, **minimize_kwargs)


Step 50; train loss 0.17399108409881592; train accuracy 95.3125; test loss 0.20855717360973358; test accuracy 93.80000233650208; alpha 0.14002800840280097; mu 0.8
Step 100; train loss 0.18654906749725342; train accuracy 92.96875; test loss 0.1707148253917694; test accuracy 94.66999769210815; alpha 0.09950371902099892; mu 0.8
Step 150; train loss 0.1538209170103073; train accuracy 95.3125; test loss 0.14442192018032074; test accuracy 95.74999809265137; alpha 0.08137884587711594; mu 0.8
Step 200; train loss 0.2395225465297699; train accuracy 91.40625; test loss 0.1320999264717102; test accuracy 95.99000215530396; alpha 0.07053456158585983; mu 0.8
Step 250; train loss 0.18236678838729858; train accuracy 94.53125; test loss 0.12463902682065964; test accuracy 96.17999792098999; alpha 0.06311944030978031; mu 0.8
Step 300; train loss 0.06560562551021576; train accuracy 99.21875; test loss 0.11419551819562912; test accuracy 96.52000069618225; alpha 0.0576390417704235; mu 0.8
Step 350; train lo

/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/opt/python/training/external_optimizer.py:402: OptimizeWarning: Unknown solver options: Hk_mat, muk
  result = scipy.optimize.minimize(*minimize_args, **minimize_kwargs)


Step 50; train loss 0.17940717935562134; train accuracy 96.875; test loss 0.25985050201416016; test accuracy 93.14000010490417; alpha 0.1614123581336696; mu 0
Step 100; train loss 0.21512523293495178; train accuracy 93.75; test loss 0.22082538902759552; test accuracy 93.84999871253967; alpha 0.0887040887040887; mu 0
Step 150; train loss 0.18039989471435547; train accuracy 96.09375; test loss 0.19883307814598083; test accuracy 94.48999762535095; alpha 0.06115623506927855; mu 0
Step 200; train loss 0.2612343430519104; train accuracy 90.625; test loss 0.18702177703380585; test accuracy 94.91999745368958; alpha 0.04666423623769595; mu 0
Step 250; train loss 0.2069089710712433; train accuracy 93.75; test loss 0.17999547719955444; test accuracy 95.06000280380249; alpha 0.03772472737989979; mu 0
Step 300; train loss 0.14704719185829163; train accuracy 96.875; test loss 0.17514127492904663; test accuracy 95.16000151634216; alpha 0.03165965866930497; mu 0
Step 350; train loss 0.2417517453432083